# Quora Question pairs


In this session we will prepare NLP's Deliverable 1.


The goal is to build a solution for:

https://www.kaggle.com/c/quora-question-pairs/overview

In [9]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
import os

In [10]:
cl = sklearn.linear_model.Perceptron()

In [11]:
ls

 Volume in drive C is OS
 Volume Serial Number is 8004-7606

 Directory of c:\Users\mecheste\Documents\GitHub\Natural-Language-Processing\nlp_deliv1_materials

12/04/2024  15:09    <DIR>          .
12/04/2024  15:09    <DIR>          ..
12/04/2024  15:09             6,148 .DS_Store
12/04/2024  15:09    <DIR>          .ipynb_checkpoints
12/04/2024  15:09            16,963 delivery_1_quora.ipynb
12/04/2024  15:09        11,734,635 quora_test_data.csv
12/04/2024  15:09        46,986,920 quora_train_data.csv
               4 File(s)     58,744,666 bytes
               3 Dir(s)  21,380,624,384 bytes free


In [12]:
path_data =  os.path.expanduser('~') 

# use this to train and VALIDATE your solution
train_df = pd.read_csv("./quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./quora_test_data.csv")

In [13]:
train_df[10:20]

,id,qid1,qid2,question1,question2,is_duplicate
10,377576,508889,508890,Where I download Bollywood movies?,From where to download Bollywood movies?,1
11,288362,264458,409304,What are the essential things to carry while t...,What are all things I should not carry to Dubai?,0
12,222625,330234,330235,What is signed char and unsigned char exactly?,How do you check if the char is signed or unsi...,0
13,201499,303523,303524,How do I write Muhammed Shamnu in Arabic?,"How can I write ""Let it be"" in arabic?",0
14,262197,201705,378466,What are the highest and average packages offe...,Are the PSUs better than MNCs?,0
15,352414,17944,283109,Is hypnotism real?,"Is Hypnotism real? If so, How to hypnotise som...",1
16,47663,85069,85070,Why do some people say vaccines cause autism?,"Do vaccines really cause autism? If so,which o...",1
17,43457,78162,78163,Do rich people worry about anything?,What do wealthy people worry about?,1
18,362478,64154,46266,How is it possible that whenever you call some...,Can someone who blocked my number see the text...,0
19,352026,480909,480910,What are the current issues impacting on opera...,What are current issues in operations management?,1


In [14]:
A_df, te_df = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)

tr_df, va_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05,random_state=123)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


Let us train a first model....

In [15]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

all_q1 = list(train_df["question1"]) + list(test_df["question1"])
all_q2 = list(train_df["question2"]) + list(test_df["question2"])
all_questions = all_q1 + all_q2

len(all_questions)

808580

The following code will not work

```python
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)
```

In [16]:
#count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
#count_vectorizer.fit(all_questions)

We got an error what should we do?

```
ValueError: np.nan is an invalid document, expected byte or unicode string.
```


#####  Exercise 1:  `cast_list_as_strings`

Build a function  **`cast_list_as_strings`** that casts each element in the input list to a string.


In [17]:
import numpy as np
types_ = [type(q).__name__ for q in all_q1]
print(np.unique(types_))

for i in range(len(all_q1)):
    if type(all_q1[i]) == float:
        print(i, all_q1[i])

['float' 'str']
9105 nan


In [18]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

In [19]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [20]:
import numpy as np
types_ = [type(q).__name__ for q in q2_train]
np.unique(types_)

array(['str'], dtype='<U3')

In [21]:
q1_train[0], q2_train[0]

('Why do I get easily bored with everything?',
 'Why do I get bored with things so quickly and easily?')


Use all the questions in train and test partitions to build a single list `all_questions` to fit the `count_vectorizer`

In [22]:
all_questions = q1_train + q2_train

In [23]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))
count_vectorizer.fit(all_questions)

CountVectorizer()

#####  Exercise 2:  `get_features_from_df`

Make a function `get_features_from_df` that given a dataframe containing the format of the training data
it returns a scipy sparse matrix with the features from question 1 and question 2 

In [24]:

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2

In [25]:
get_features_from_df(train_df, count_vectorizer)

<323432x156550 sparse matrix of type '<class 'numpy.int64'>'
	with 6494394 stored elements in Compressed Sparse Row format>

In [26]:
X_tr_q1q2 = get_features_from_df(train_df, count_vectorizer)
X_te_q1q2  = get_features_from_df(test_df, count_vectorizer)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

((323432, 156550), (323432, 6), (80858, 6), (80858, 156550))

Now You can use this representation `X_tr_q1q2` to fit a model

In [27]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
y_train = train_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

LogisticRegression(random_state=123, solver='liblinear')

## Examine mistakes


#####  Exercise 3:  `get_mistakes`

Make a function `get_mistakes` that given a model `clf` a dataframe `df`, the features `X_q1q2` and the target labels `y`returns 


- incorrect_indices: coordinates where the model made a mistake
- predictions: predictions made by the model


In [28]:
def get_mistakes(clf, X_q1q2, y):

    ############### Begin exercise ###################
    predictions = clf.predict(X_q1q2)
    incorrect_predictions = predictions != y 
    incorrect_indices,  = np.where(incorrect_predictions)
    
    ############### End exercise ###################
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions

In [29]:
y_train = train_df["is_duplicate"].values

mistake_indices, predictions = get_mistakes(logistic,
                                            X_tr_q1q2, 
                                            y_train)

In [30]:
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])

In [31]:
#[x for x in X if "'s" in x]
print_mistake_k(100, mistake_indices, predictions)

What is the best civil engineering company for a job?
What are the best companies for a civil engineer?
true class: 1
prediction: 0


In [32]:
print_mistake_k(12, mistake_indices, predictions)

Why is it so believed that aliens (if they exist) will be smarter than us human beings?
Is God an alien more intelligent than humans?
true class: 0
prediction: 1


In [33]:
# w1 w2 ... wn
# v1 v2 ... vm

In [34]:
# w1vec = word2vec.transform(w1)
# len(w1vec) = 256

# w2vec = word2vec.transform(w2)
# len(w2vec) = 256

# EmbA  = w1vec + ... + wnvec -> 256
# EmbB  = v1vec + ... + vnvec -> 256
# concat(256, 256)            -> 512
# word2vec feature: d(EmbA, EmbB)

# Improving feature vectors:

Build a Similarity metric between documents using the tf-idf vectors of the documents.

Build a class from scratch that can learn a tf-idf feature representation.